In [1]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

2022-05-14 18:09:41.538751: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-14 18:09:41.538839: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train / 255
X_test = X_test / 255

y_test = to_categorical(y_test, num_classes = 10)
y_train = to_categorical(y_train, num_classes = 10)

dimensions = X_train.shape[1] * X_train.shape[2]

X_train = tf.reshape(X_train, (X_train.shape[0], dimensions))
X_test = tf.reshape(X_test, (X_test.shape[0], dimensions))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

2022-05-14 18:09:56.498499: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-14 18:09:56.498649: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-14 18:09:56.498732: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debonair): /proc/driver/nvidia/version does not exist
2022-05-14 18:09:56.531296: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-14 18:09:56.597060: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 376320000 exceeds 

(TensorShape([60000, 784]),
 TensorShape([10000, 784]),
 (60000, 10),
 (10000, 10))

In [3]:
def build_model(hp):
    model = tf.keras.Sequential([
        #tf.keras.Input(shape = (None, 28, 28)),
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(10, hp.Choice('activation', ['sigmoid', 'softmax']))
    ])
    
    #Compile model.
    model.compile(loss = 'categorical_crossentropy',
                  metrics = ['accuracy'],
                  optimizer = 'adam')
    return model

In [4]:
!rm -r 'untitled_project/'

In [5]:
tuner = kt.RandomSearch(
    build_model, 
    objective = 'accuracy',
    max_trials = 5)

In [6]:
tuner.search(X_train, y_train, epochs = 5, verbose = 1, validation_data = (X_test, y_test))


Trial 2 Complete [00h 00m 38s]
accuracy: 0.9799500107765198

Best accuracy So Far: 0.9799500107765198
Total elapsed time: 00h 01m 31s
INFO:tensorflow:Oracle triggered exit


In [7]:
best_model_hparam = tuner.get_best_hyperparameters()
best_model = tuner.get_best_models(num_models = 2)

In [8]:
best_model_hparam

In [9]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
activation: softmax
Score: 0.9799500107765198
Trial summary
Hyperparameters:
activation: sigmoid
Score: 0.9792500138282776
